Import Needed Libraries

In [3]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import matplotlib
import xlrd
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
import random
seed_value =10
random.seed(seed_value)
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
from tensorflow import set_random_seed
set_random_seed(seed_value)
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation, Dropout,RepeatVector,TimeDistributed
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import load_model

Using TensorFlow backend.


In [4]:
def get_dataset_for_variable(variables,excel_path):
    
    month_convert={
            4:"Arp",
            5:"May",
            6:"Jun",
            7:"Jul",
            8:"Aug",
            9:"Sep",
            10:"Oct",
            11:"Nov",
            12:"Dec",
            1:"Jan",
            2:"Feb",
            3:"Mar"
        }
    workbook = xlrd.open_workbook(excel_path) #read input
    worksheet = workbook.sheet_by_index(0)

    keys={}
    keys["Month"]=[]

    #find all keys
    forbid={}
    for i in range(3,worksheet.nrows):#for each row
        forbid[i]=1
        for var in variables:
            if var.lower() in str(worksheet.cell(i,5).value).lower(): #column3 is product name and column 5 is variable
                key=str(worksheet.cell(i,5).value)+"-"+str(worksheet.cell(i,3).value)
                keys[key]=[]
                break

    j=9 #date column begin from 9 in format

    while j<worksheet.ncols:
        date3=str(xlrd.xldate.xldate_as_datetime(worksheet.cell(2,j).value,workbook.datemode))
        final_date=date3[5]+date3[6]+"-"+date3[8]+date3[9]+"-"+date3[0]+date3[1]+date3[2]+date3[3]
        ok=0
        for i in range(3,worksheet.nrows):#for each row
            bol=str(type(worksheet.cell(i,j).value))=="<class 'float'>"
            if str(worksheet.cell(i,j).value)=='x':
                forbid[i]=0
            if bol==1:
                for var in variables:
                    if var.lower() in str(worksheet.cell(i,5).value).lower():
                        key=str(worksheet.cell(i,5).value)+"-"+str(worksheet.cell(i,3).value)
                        keys[key].append(worksheet.cell(i,j).value)
                        ok=1
                        break
            elif forbid[i]:
                for var in variables:
                    if var.lower() in str(worksheet.cell(i,5).value).lower():
                        key=str(worksheet.cell(i,5).value)+"-"+str(worksheet.cell(i,3).value)
                        keys[key].append(0.0)
                        ok=1
                        break
        if ok==1:
            keys["Month"].append(final_date)
        j+=1

    #convert it to proper pandas format
    colname=[]
    leng=0
    for key in keys:
        colname.append(key)
        leng=len(keys[key])
    datarow=[]
    for w in range(0,leng):
        rw=[]
        for key in keys:
            rw.append(keys[key][w])
        rw.append(float(w))
        datarow.append(rw)
    colname.append('Index')
    dataset = pd.DataFrame(datarow,columns=colname).set_index('Index')
    return dataset

In [5]:
def get_dataset_for_product(dataset,column_name):
    dataset=pd.DataFrame(dataset[column_name])
    return dataset,column_name

In [6]:
def normalize_dataset(dataset,column_name):
    dataframe = dataset
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    scalerfile = os.path.join(model_path,'scaler-%s.sav'%column_name)
    pickle.dump(scaler, open(scalerfile, 'wb'))
    return dataset,scaler

In [7]:
def create_dataset(dataset, look_back=12):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [26]:
import os
model_path ="models"
if not os.path.exists(model_path):
    os.makedirs(model_path)

In [27]:
## creating a dataset with needed columns and its values
variables=["Total Revenue"] # variable name
excel_path = 'PrivatizedDataforParticipants.xlsx'
dataset = get_dataset_for_variable(variables,excel_path)

In [28]:
column_name = 'Total Revenue-Panther - Leopard - Lion'# product_key_name
dataset,column_name = get_dataset_for_product(dataset,column_name)

In [29]:
datasetx=dataset

In [30]:
dataset,scaler = normalize_dataset(dataset,column_name)

In [31]:
dataset.shape

(42, 1)

In [32]:
look_back=6 # chnage look back to 12 or other values here
trainX, trainY = create_dataset(dataset, look_back=look_back)

In [33]:
# reshape input to be [samples, features, time steps]
trainX = np.reshape(trainX, (trainX.shape[0],trainX.shape[1], 1))
trainY = np.reshape(trainY, (trainY.shape[0],1, 1))

In [34]:
# training data spli
valX = trainX[-look_back:]
trainX = trainX[:-look_back]

valY = trainY[-look_back:]
trainY = trainY[:-look_back]


In [35]:
trainX.shape,trainY.shape

((30, 6, 1), (30, 1, 1))

In [36]:
valX.shape,valY.shape

((6, 6, 1), (6, 1, 1))

In [37]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(30, activation='relu'))
model.add(RepeatVector(1))
model.add(TimeDistributed(Dense(1, activation="linear")))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 6, 50)             10400     
_________________________________________________________________
lstm_5 (LSTM)                (None, 6, 50)             20200     
_________________________________________________________________
lstm_6 (LSTM)                (None, 30)                9720      
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 1, 30)             0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 1, 1)              31        
Total params: 40,351
Trainable params: 40,351
Non-trainable params: 0
_________________________________________________________________


In [38]:
epochs =500 # no of epochs
bs=6 # batch size
model.fit(trainX, trainY, epochs=epochs, batch_size=bs, verbose=1)

Epoch 1/500
30/30 [==============================] - 2s 78ms/step - loss: 0.4017 - mean_squared_error: 0.4017
Epoch 2/500
30/30 [==============================] - 0s 2ms/step - loss: 0.3665 - mean_squared_error: 0.3665
Epoch 3/500
30/30 [==============================] - 0s 3ms/step - loss: 0.3259 - mean_squared_error: 0.3259
Epoch 4/500
30/30 [==============================] - 0s 4ms/step - loss: 0.2701 - mean_squared_error: 0.2701
Epoch 5/500
30/30 [==============================] - 0s 4ms/step - loss: 0.1967 - mean_squared_error: 0.1967
Epoch 6/500
30/30 [==============================] - 0s 3ms/step - loss: 0.1124 - mean_squared_error: 0.1124
Epoch 7/500
30/30 [==============================] - 0s 3ms/step - loss: 0.0215 - mean_squared_error: 0.0215
Epoch 8/500
30/30 [==============================] - 0s 5ms/step - loss: 0.0257 - mean_squared_error: 0.0257
Epoch 9/500
30/30 [==============================] - 0s 4ms/step - loss: 0.0166 - mean_squared_error: 0.0166
Epoch 10/500
30/30

30/30 [==============================] - 0s 3ms/step - loss: 6.2213e-04 - mean_squared_error: 6.2213e-04
Epoch 72/500
30/30 [==============================] - 0s 3ms/step - loss: 6.2943e-04 - mean_squared_error: 6.2943e-04
Epoch 73/500
30/30 [==============================] - 0s 2ms/step - loss: 7.8509e-04 - mean_squared_error: 7.8509e-04
Epoch 74/500
30/30 [==============================] - 0s 2ms/step - loss: 7.0759e-04 - mean_squared_error: 7.0759e-04
Epoch 75/500
30/30 [==============================] - 0s 2ms/step - loss: 6.7683e-04 - mean_squared_error: 6.7683e-04
Epoch 76/500
30/30 [==============================] - 0s 3ms/step - loss: 6.5706e-04 - mean_squared_error: 6.5706e-04
Epoch 77/500
30/30 [==============================] - 0s 3ms/step - loss: 6.4081e-04 - mean_squared_error: 6.4081e-04
Epoch 78/500
30/30 [==============================] - 0s 2ms/step - loss: 6.7307e-04 - mean_squared_error: 6.7307e-04
Epoch 79/500
30/30 [==============================] - 0s 2ms/step - l

30/30 [==============================] - 0s 2ms/step - loss: 6.9095e-04 - mean_squared_error: 6.9095e-04
Epoch 141/500
30/30 [==============================] - 0s 2ms/step - loss: 7.1910e-04 - mean_squared_error: 7.1910e-04
Epoch 142/500
30/30 [==============================] - 0s 2ms/step - loss: 8.2333e-04 - mean_squared_error: 8.2333e-04
Epoch 143/500
30/30 [==============================] - 0s 2ms/step - loss: 6.0467e-04 - mean_squared_error: 6.0467e-04
Epoch 144/500
30/30 [==============================] - ETA: 0s - loss: 4.7735e-04 - mean_squared_error: 4.7735e- - 0s 2ms/step - loss: 6.7697e-04 - mean_squared_error: 6.7697e-04
Epoch 145/500
30/30 [==============================] - 0s 2ms/step - loss: 6.8074e-04 - mean_squared_error: 6.8074e-04
Epoch 146/500
30/30 [==============================] - 0s 2ms/step - loss: 6.3803e-04 - mean_squared_error: 6.3803e-04
Epoch 147/500
30/30 [==============================] - 0s 2ms/step - loss: 6.4454e-04 - mean_squared_error: 6.4454e-04
Ep

30/30 [==============================] - 0s 2ms/step - loss: 7.3500e-04 - mean_squared_error: 7.3500e-04
Epoch 209/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0010 - mean_squared_error: 0.0010    
Epoch 210/500
30/30 [==============================] - 0s 2ms/step - loss: 6.3772e-04 - mean_squared_error: 6.3772e-04
Epoch 211/500
30/30 [==============================] - 0s 2ms/step - loss: 6.9615e-04 - mean_squared_error: 6.9615e-04
Epoch 212/500
30/30 [==============================] - 0s 2ms/step - loss: 5.9497e-04 - mean_squared_error: 5.9497e-04
Epoch 213/500
30/30 [==============================] - 0s 2ms/step - loss: 6.4982e-04 - mean_squared_error: 6.4982e-04
Epoch 214/500
30/30 [==============================] - 0s 3ms/step - loss: 6.3251e-04 - mean_squared_error: 6.3251e-04
Epoch 215/500
30/30 [==============================] - 0s 3ms/step - loss: 8.4188e-04 - mean_squared_error: 8.4188e-04
Epoch 216/500
30/30 [==============================] - 0s 2ms/step

30/30 [==============================] - 0s 2ms/step - loss: 5.8297e-04 - mean_squared_error: 5.8297e-04
Epoch 278/500
30/30 [==============================] - 0s 3ms/step - loss: 7.8761e-04 - mean_squared_error: 7.8761e-04
Epoch 279/500
30/30 [==============================] - 0s 2ms/step - loss: 8.2599e-04 - mean_squared_error: 8.2599e-04
Epoch 280/500
30/30 [==============================] - 0s 3ms/step - loss: 6.1505e-04 - mean_squared_error: 6.1505e-04
Epoch 281/500
30/30 [==============================] - 0s 2ms/step - loss: 5.9179e-04 - mean_squared_error: 5.9179e-04
Epoch 282/500
30/30 [==============================] - 0s 2ms/step - loss: 7.4425e-04 - mean_squared_error: 7.4425e-04
Epoch 283/500
30/30 [==============================] - 0s 2ms/step - loss: 8.7604e-04 - mean_squared_error: 8.7604e-04
Epoch 284/500
30/30 [==============================] - 0s 2ms/step - loss: 6.9236e-04 - mean_squared_error: 6.9236e-04
Epoch 285/500
30/30 [==============================] - 0s 3ms/

30/30 [==============================] - 0s 2ms/step - loss: 5.9140e-04 - mean_squared_error: 5.9140e-04
Epoch 347/500
30/30 [==============================] - 0s 2ms/step - loss: 6.4970e-04 - mean_squared_error: 6.4970e-04
Epoch 348/500
30/30 [==============================] - 0s 2ms/step - loss: 6.6556e-04 - mean_squared_error: 6.6556e-04
Epoch 349/500
30/30 [==============================] - 0s 2ms/step - loss: 7.1691e-04 - mean_squared_error: 7.1691e-04
Epoch 350/500
30/30 [==============================] - 0s 2ms/step - loss: 9.3996e-04 - mean_squared_error: 9.3996e-04
Epoch 351/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0011 - mean_squared_error: 0.0011    
Epoch 352/500
30/30 [==============================] - 0s 2ms/step - loss: 5.3513e-04 - mean_squared_error: 5.3513e-04
Epoch 353/500
30/30 [==============================] - 0s 2ms/step - loss: 9.3125e-04 - mean_squared_error: 9.3125e-04
Epoch 354/500
30/30 [==============================] - 0s 3ms/step

30/30 [==============================] - 0s 2ms/step - loss: 8.8304e-04 - mean_squared_error: 8.8304e-04
Epoch 416/500
30/30 [==============================] - 0s 2ms/step - loss: 9.2148e-04 - mean_squared_error: 9.2148e-04
Epoch 417/500
30/30 [==============================] - 0s 2ms/step - loss: 8.3955e-04 - mean_squared_error: 8.3955e-04
Epoch 418/500
30/30 [==============================] - 0s 2ms/step - loss: 7.8283e-04 - mean_squared_error: 7.8283e-04
Epoch 419/500
30/30 [==============================] - 0s 2ms/step - loss: 5.7964e-04 - mean_squared_error: 5.7964e-04
Epoch 420/500
30/30 [==============================] - 0s 2ms/step - loss: 6.5576e-04 - mean_squared_error: 6.5576e-04
Epoch 421/500
30/30 [==============================] - 0s 2ms/step - loss: 6.4892e-04 - mean_squared_error: 6.4892e-04
Epoch 422/500
30/30 [==============================] - 0s 2ms/step - loss: 5.9095e-04 - mean_squared_error: 5.9095e-04
Epoch 423/500
30/30 [==============================] - 0s 2ms/

30/30 [==============================] - 0s 2ms/step - loss: 7.8098e-04 - mean_squared_error: 7.8098e-04
Epoch 485/500
30/30 [==============================] - 0s 2ms/step - loss: 7.7569e-04 - mean_squared_error: 7.7569e-04
Epoch 486/500
30/30 [==============================] - 0s 2ms/step - loss: 7.8995e-04 - mean_squared_error: 7.8995e-04
Epoch 487/500
30/30 [==============================] - 0s 2ms/step - loss: 8.3348e-04 - mean_squared_error: 8.3348e-04
Epoch 488/500
30/30 [==============================] - 0s 2ms/step - loss: 8.0846e-04 - mean_squared_error: 8.0846e-04
Epoch 489/500
30/30 [==============================] - 0s 2ms/step - loss: 6.5715e-04 - mean_squared_error: 6.5715e-04
Epoch 490/500
30/30 [==============================] - 0s 2ms/step - loss: 9.2138e-04 - mean_squared_error: 9.2138e-04
Epoch 491/500
30/30 [==============================] - 0s 2ms/step - loss: 6.4061e-04 - mean_squared_error: 6.4061e-04
Epoch 492/500
30/30 [==============================] - 0s 2ms/

In [39]:
## cross validation
def predict_values(testX,model,scaler):  
    testX = np.reshape(testX, (1,testX.shape[0], 1))
    testPredict = model.predict(testX)        
    testPredict = np.array(testPredict[0]).flatten().reshape(-1, 1)
    output = scaler.inverse_transform(testPredict).flatten()
    output = output[0].tolist()
    return output

In [40]:
# MAPE score calculation
def cal_mape(ypred,y):
    mape = max(0, 1 - abs(y-ypred)/abs(y))
    return mape

scores =[]
for i in range(valX.shape[0]):
    y_pred = predict_values(valX[i],model,scaler)
    print(y_pred,scaler.inverse_transform(valY[i])[0][0])
    scores.append(cal_mape(y_pred,scaler.inverse_transform(valY[i])[0][0]))
print(100 * sum(scores)/len(scores))

0.8490790724754333 0.7795444
0.8835713267326355 0.91879356
0.9455041885375977 0.9033173
0.9856700897216797 1.0556935
1.0484764575958252 1.0867313
1.1665549278259277 1.0900182
94.23360431197885


In [41]:
model_save_path = os.path.join(model_path,column_name+".h5")
scaler_save_path = os.path.join(model_path,'scaler-%s.sav'%column_name)

In [42]:
model.save(model_save_path)

In [43]:
## final prediction
def predict_values(testX,no_of_pred,model,scaler):
    final_output = []
    for i in range(0,no_of_pred):
        testX = np.reshape(testX, (1,testX.shape[0], 1))
        testPredict = model.predict(testX)  
        testX = np.reshape(testX, (testX.shape[1], 1))
        testX = np.concatenate((testX[1:],testPredict[0]))
        testPredict = np.array(testPredict[0]).flatten().reshape(-1, 1)
        output = scaler.inverse_transform(testPredict).flatten()
        output = output[0].tolist()
        final_output.append(output)
    return final_output

In [44]:
no_of_pred = 6 # forecast_horizon
testX = datasetx.iloc[-no_of_pred:].values
scaler = pickle.load(open(scaler_save_path, 'rb'))
testX = scaler.transform(np.array(testX).astype('float32').reshape(-1, 1))

In [45]:
from keras.models import load_model
model = load_model(model_save_path)
final_output = predict_values(testX,no_of_pred,model,scaler)
final_output

[1.1826105117797852,
 1.2627555131912231,
 1.3192580938339233,
 1.4020986557006836,
 1.4574819803237915,
 1.5248396396636963]